In [40]:
# import libraries
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import load_img, img_to_array


In [26]:
# configurations
IMG_SIZE = 64
BATCH_SIZE = 32
EPOCHS = 10

DATASET_DIR = '../data/breast_cancer_complete_set'


In [35]:
# load the data 
training_dir = os.path.join(DATASET_DIR, 'training_set')
testing_dir = os.path.join(DATASET_DIR, 'testing_set')
testing_dir

'../data/breast_cancer_complete_set\\testing_set'

In [29]:
# convert images to grayscale
train_datagen = ImageDataGenerator(rescale=1./255)
testing_datagen = ImageDataGenerator(rescale=1./255)

In [47]:
training_generator = train_datagen.flow_from_directory(
    training_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    color_mode='grayscale',
    subset='training',
    shuffle=True
)

def load_testing_images(testing_dir, img_size):
    image_list = []
    image_names = []

    for fname in os.listdir(testing_dir):
        fpath = os.path.join(testing_dir, fname)
        if fname.lower().endswith(('.png', '.jpg', '.jpeg')):  # filter out non-image files
            img = load_img(fpath, color_mode='grayscale', target_size=(img_size, img_size))
            img_array = img_to_array(img) / 255.0  # normalize
            image_list.append(img_array)
            image_names.append(fname)

    return np.array(image_list), image_names

testing_data, image_names = load_testing_images(testing_dir, IMG_SIZE)

Found 1112 images belonging to 2 classes.


In [48]:
testing_data[:5]

array([[[[0.62352943],
         [0.68235296],
         [0.69411767],
         ...,
         [0.49411765],
         [0.56078434],
         [0.63529414]],

        [[0.6862745 ],
         [0.654902  ],
         [0.6509804 ],
         ...,
         [0.7764706 ],
         [0.654902  ],
         [0.6862745 ]],

        [[0.9137255 ],
         [0.80784315],
         [0.8392157 ],
         ...,
         [0.8235294 ],
         [0.8235294 ],
         [0.8117647 ]],

        ...,

        [[0.08627451],
         [0.0627451 ],
         [0.08235294],
         ...,
         [0.07058824],
         [0.04313726],
         [0.04705882]],

        [[0.01960784],
         [0.        ],
         [0.05098039],
         ...,
         [0.08627451],
         [0.09019608],
         [0.08235294]],

        [[0.06666667],
         [0.0627451 ],
         [0.08235294],
         ...,
         [0.05490196],
         [0.09411765],
         [0.04705882]]],


       [[[0.        ],
         [0.        ],
         [0.  

In [49]:
# model architecture
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


c:\Users\Dee\Desktop\My_Projects\ml_projects\plp_final_project\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 62, 62, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     1,605,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,624,705 (6.20 MB)

 Trainable params: 1,624,705 (6.20 MB)

 Non-trainable params: 0 (0.00 B)

In [50]:
# train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit(
    training_generator,
    epochs=EPOCHS,
    validation_data=(testing_data, np.zeros(len(testing_data))),
    callbacks=[early_stopping]
)

c:\Users\Dee\Desktop\My_Projects\ml_projects\plp_final_project\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 19s 410ms/step - accuracy: 0.6977 - loss: 0.6298 - val_accuracy: 0.9950 - val_loss: 0.2620
Epoch 2/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 14s 401ms/step - accuracy: 0.7309 - loss: 0.5640 - val_accuracy: 0.8600 - val_loss: 0.4663
Epoch 3/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 15s 413ms/step - accuracy: 0.7541 - loss: 0.4990 - val_accuracy: 0.7600 - val_loss: 0.5617
Epoch 4/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 14s 410ms/step - accuracy: 0.7663 - loss: 0.4706 - val_accuracy: 0.7600 - val_loss: 0.5551


In [51]:
# evaluate the model
test_loss, test_accuracy = model.evaluate(testing_data, np.zeros(len(testing_data)))
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.9950 - loss: 0.2565
Test Loss: 0.26202312111854553, Test Accuracy: 0.9950000047683716


In [53]:
# save the model
model.save('../models/breast_cancer_classifier.h5')
print("Model saved as 'breast_cancer_classifier.h5'")

Model saved as 'breast_cancer_classifier.h5'
